<a href="https://colab.research.google.com/github/bodorcy/hazifeladatok/blob/main/ml_2_gyakorlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Egyszerű döntési szabályok - gyakorlás

Az [előző órán](https://colab.research.google.com/drive/1X1MWF3ODdGoJWF9a8QQUQf8rN-W3l0z3) már megismert Titanic adatbázist fogjuk használni.



In [1]:
# openml-hez szükséges csomag
from sklearn.datasets import fetch_openml
# Később még szükség lesz rá
import pandas as pd

# Adatbázis betöltése az OpenML-ből
titanic = fetch_openml(name="titanic", version=1, as_frame=True)

# Jellemzővektorok Pandas DataFrame formátumban
df = titanic.data

# A célváltozót (y) is hozzárendeljük
df["survived"] = titanic.target

# Alapstatisztikák
Határozd meg folytonos változók alapstatisztikáit (átlag, szórás, min, max, medián).

Határozd meg, hogy az utasok neme milyen kapcsolatban van a túléléssel.

# Vizualizáció

Jelenítsd meg az életkor histogramját. Az intervallumként használt oszlopok száma legyen 20.

Ábrázold az életkor és a jegyárak kapcsolat.
 - Csak a 300 dollár alatti jegyárakat vedd figyelembe. (A kiugró értékek elrontják az ábrát.)
 - Készíts egy scatter plot-ot ez alapjá.

Van-e összefüggés a kor és a jegyár között?

Ábrázold, hogy az eltérő utasosztályokon (pclass) milyen volt az emberek korának az eloszlása.

Ehhez használj violin plotot, ami az adatokat egyszerre ábrázolja egy folytonos hisztogram és egy box plot formájában.

Violin plot készítésére a legegyszerűbben módszer a [seaborn könyvtár](https://seaborn.pydata.org/generated/seaborn.violinplot.html) használata.

Fontos paraméterek:
 - data: a DataFrame, amit használni akarunk
 - x: oszlopnév: ami alapján csoportosítani szeretnénk
 - y: oszlopnév: aminek az eloszlását meg akarjuk jeleníteni

Példa:

``` python
import seaborn as sns

sns.violinplot(x='...', y='...', data=...)
```
![link text](https://seaborn.pydata.org/archive/0.11/_images/seaborn-violinplot-2.png)

Van-e kapcsolat az utasosztály és az életkor között?

# Döntési szabályok - Osztályozás

Legyen a túlélés a célváltozó, amit predikálni szeretnénk.

Készíts egy döntési szabályt, ami azt mondja, hogy mindenki, aki az elsőosztályon tartózkodott túlélte az utat, mindenki más meghalt.

Mekkora az így kapott rendszer accuracy-je?

Mekkora lenne az accuracy, ha mindig a leggyakoribb osztályt választanánk?

Készíts saját döntési szabályokat az osztályozási feladatra. Légy kreatív.

Mi a legmagasabb accuracy, amit el tudtál érni?

# Döntési szabályok - Regresszió

Legyen a célváltozó a jegyár, próbálj minél jobb döntési szabályt készíteni a többi jellenző alapján.

Készítsétek el az alábbi döntési szabályt:
 - a jegyár legyenaz adott utasosztály jegyárainak az átlaga.

Értékeljétek ki a Mean Absolute Error-t.

# Jellemző kiválasztás

A lenti kódrészlet az adatbázisunkat szétszedi jellemzőkre és egy osztálycímkére (túlélte-e a katasztrófát).

A jellemzőket hozzátok gépi tanuló rendszerek által emészthető formába, ehhez:
 - alakítsátok numerikussá az adatot one hot encoding segítségével
 - a hiányzó adatokat töltsétek fel 0-kkal

Határozzátok meg a Chi2 statisztika melyik jellemzőt tartja a legfontosabbnak?

In [ ]:
features = df[['pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare',
             'cabin', 'embarked', 'boat', 'body', 'home.dest']]
label = df['survived']
